# ITNPBD2 OpenCV in Python
## Image manipulation and object detection

# pip install opencv-python
We also need matplotlib to display the images in the notebook

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

# Images are stored in NumPy arrays

## Let's make one

In [ ]:
img = [[[x,x,x] for x in range(256)] for i in range(256)]
img=np.array(img).astype(np.uint8)
img[:,:,0]=255-img[:,:,0]
img[:,:,1]=0

#cv2.imshow('image',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

plt.imshow(img)
plt.title('Image')
plt.show()

#print(img)

## Load an image

In [ ]:
img = cv2.imread('Images/mountain.jpg')
plt.imshow(img)
plt.title('Bannf')
plt.show()

## The colours don't look right!
matplotlib expects RGB but opencv uses BGR as default, so we translate:
I'll remove the axis lines and scale too

In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title('Bannf')
plt.axis('off')
plt.show()

# What is stored in the variable `img`?
## It is a NumPy array of integers: h x w x 3 colour channels:

In [ ]:
print(type(img))
print(img.shape)
print(img)

# Lets take a closer look at the NumPy array
- The shape is (rows, columns, 3)
- Access a single pixel at `img[y,x]` - note the order row, column
- Origin (0,0) is at top left of image
- This gives the three colour channels. By default, RGB (if viewing with matplotlib)
- So red intensity of pixel in row 5, column 10 is at `img[5,10,0]`, green is at `img[5,10,1]` and blue is at `img[5,10,2]`

In [ ]:
fade = [[[x,x,x] for x in range(256)] for i in range(256)]
fade=np.array(fade).astype(np.uint8)
fade[:,:,0]=255-fade[:,:,0]
fade[:,:,1]=0

'''
# This shows the image in a new window and uses BGR colour scheme!!!
cv2.imshow("test", fade)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

plt.imshow(fade)
plt.title('Fade')
plt.axis('off')
plt.show()


In [ ]:
print("Red intensity at 0,0 =", fade[0,0,0])
print("Red intensity at 0,255 =", fade[0,255,0])
print("Blue intensity at 0,0 =", fade[0,0,2])
print("Blue intensity at 0,255 =", fade[0,255,2])

## Scaling and Rotating

In [ ]:
height,width,_ = img.shape
bigger = cv2.resize(img,(2*width, 2*height), interpolation = cv2.INTER_CUBIC)
print(bigger.shape)

In [ ]:
rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), 90, 1)  # or -1 for other way
rotated_image = cv2.warpAffine(img, rotation_matrix, (width, height))
plt.imshow(rotated_image)
plt.title('Bannf')
plt.axis('off')
plt.show()

# Drawing on the image

In [ ]:
cv2.circle(img,(100,100),50,(250,20,20),5)   # Make negative thickness to fill
plt.imshow(img)
plt.title('Bannf')
plt.axis('off')
plt.show()

# Detecting Edges and Lines

In [ ]:
img = cv2.imread('Images/parrot.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title('Parrot')
plt.show()

In [ ]:
# Canny edge detection
threshold1 = 100
threshold2 = 200
canny = cv2.Canny(img, threshold1, threshold2)

plt.imshow(canny, cmap='gray')

# Blurring and other filters

In [ ]:
blur=cv2.blur(img,(5,5))
plt.imshow(blur)

In [ ]:
canny = cv2.Canny(blur, threshold1, threshold2)

plt.imshow(canny, cmap='gray')

In [ ]:
median = cv2.medianBlur(img,15)
plt.imshow(median)

In [ ]:
canny = cv2.Canny(median, threshold1, threshold2)

plt.imshow(canny, cmap='gray')

# Finding Lines in an Image
- Hough Transform finds lines and returns an array of polar coordinates
- They are rho - distance from origin and theta - angle of perpendiular line to origin
- Doesn't give the length of the line
- Need to translate them into x,y coordinates for plotting on image

<img src="Images/houghlines1.svg">

In [ ]:
img = cv2.imread('Images/window.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
plt.imshow(img)

In [ ]:
img = cv2.imread('Images/window.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blur = cv2.blur(gray,(2,2))
edges = cv2.Canny(blur,50,150,apertureSize = 3)
lines = cv2.HoughLines(edges,1,0.001,170)
maxdim = max(img.shape)

print("Found",len(lines),"lines")

def polar_to_cart(rho,theta, length):
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + length*(-b))
    y1 = int(y0 + length*(a))
    x2 = int(x0 - length*(-b))
    y2 = int(y0 - length*(a))
    return x1, x2, y1, y2

for line in lines:
    for rho,theta in line:
        #print("Line: rho=",rho,"theta=",theta)
        x1, x2, y1, y2 = polar_to_cart(rho,theta, maxdim)
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),3)

plt.imshow(img)

# Filtering by Colour

In [ ]:
img = cv2.imread('Images/apple.jpg')
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(rgb)
plt.title('Apple')
plt.show()

# Translate the image into Hue Saturation, Value (HSV) space for colour processing
## Good way to understand HSV space, use a colour chooser like the one in MS Paint ...

In [ ]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
plt.imshow(hsv)

## Mask out the regions that are red in HSV space

In [ ]:
# In HSV colour space, red occupies two main regions:

# lower mask (0-10)
lower_red = np.array([0,90,70])
upper_red = np.array([10,255,255])
mask0 = cv2.inRange(hsv, lower_red, upper_red)

# upper mask (170-180)
lower_red = np.array([170,120,70])
upper_red = np.array([180,255,255])
mask1 = cv2.inRange(hsv, lower_red, upper_red)

# join my masks
mask = mask0+mask1
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((6,6),np.uint8),iterations=2) # Remove dots

# Convert colour space to grayscale
#cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
#cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.imshow(mask, cmap='gray')

## Calculate the image moments
* Image moments are statistics calculated from the image intensity values
* We will use them to calculate a centre of mass for the image

In [ ]:
m = cv2.moments(mask, 0)

cx = int(m['m10']/m['m00'])
cy = int(m['m01']/m['m00'])

print(cx,cy)

img_marked = rgb.copy()
cv2.circle(img_marked,(cx,cy),60,(250,20,20),-1)
plt.imshow(img_marked)

# Now Calculate the image contours
## Edge detection produces a new image where edges are a different colour
## Contour calculation produces a list of points that make up a contour- an edge at the same height
## There will be many in an image, so an array of countour sets is produced when calling `cv2.findContours`

In [ ]:
contours,_ = cv2.findContours(mask, 1, 2)
print(len(contours))
#cv2.drawContours(img, contours, -1, (0, 255, 0), 15)
#plt.imshow(img)

# minEnclosingCircle finds the smallest circle that encloses a contour
# Here we find the largest and hope that is our apple
maxr=0
for cnt in contours:
    (x,y),radius = cv2.minEnclosingCircle(cnt)
    centre = (int(x),int(y))
    radius = int(radius)
    if(radius>maxr):
        maxr = radius
        maxc = centre
if maxr > 0:
    cv2.circle(img_marked,maxc,maxr,(255,0,0),20)


plt.imshow(img_marked)
print(x,y,radius)

# Apply a mask to select only the pixels in the circle of the apple
* Make a new image the same size as the first one
* Create a mask for the circle around the apple
* Use the mask to create two new images: apple, background

In [ ]:
height,width,_ = img.shape
applemask = np.zeros((height,width), np.uint8)
cv2.circle(applemask,maxc,maxr,(255,255,255),-1)
bgmask = cv2.bitwise_not(applemask)
plt.imshow(bgmask,cmap='gray')

In [ ]:
# Mask and add
bg = cv2.bitwise_and(rgb,rgb,mask=bgmask)
apple = cv2.bitwise_and(rgb,rgb,mask=applemask)
plt.imshow(apple)

# Finally, blur the background and add the images together

In [ ]:
bgblur = cv2.blur(img,(60, 60))
bgblur = cv2.bitwise_and(bgblur, bgblur, mask=bgmask)
final=bgblur+apple
plt.imshow(final)

# Detecting Faces
## Using Haar Cascades
## Open CV comes with a set of detectors for facial features
## They are defined in XML files, usually here:
`...\Lib\site-packages\cv2\data`
## Detectors for:
* Eyes
* Faces
* Smiles
* Upper / Lower body

## Read about them here:
<a href='https://docs.opencv.org/4.1.0/d7/d8b/tutorial_py_face_detection.html'>https://docs.opencv.org/4.1.0/d7/d8b/tutorial_py_face_detection.html</a>

In [ ]:
img = cv2.imread('Images/prize.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

classifier = cv2.CascadeClassifier("haar/haarcascade_frontalface_default.xml")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = classifier.detectMultiScale(gray, 1.1, 5)
for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    
plt.imshow(img)